In [ ]:
include("main.jl")
using PyCall
using CairoMakie
sepsis_gym = pyimport("dbn_ppl_rl")
using Statistics


In [8]:
struct MeansType
    individual_runs::Vector{Vector{Float64}}
    mean_rewards::Vector{Float64}
    smoothed_mean::Vector{Float64}
    smoothed_std_low::Vector{Float64}
    smoothed_std_high::Vector{Float64}
    keys_of_smoothed::Vector{Int64}
end


In [ ]:
ts = Dict(
    :Simple100 => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/Simple-$i.json") for i in 0:2],
    :Medium100 => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/Medium-$i.json") for i in 0:2],
    :None100P => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/None-small-prior-$i.json") for i in 0:2],
    :None100 => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/None-$i.json") for i in 0:2],
    :Simple1 => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/Simple-every-$i.json") for i in 0:2],
    :Medium1 => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/Medium-every-$i.json") for i in 0:2],
    :None1P => [sepsis_gym.DirThompsonSampling.load_json("data/raw/dbn/None-small-prior-every-$i.json") for i in 0:2],
    :Softmax100 => [load_jld("data/raw/ppl/SoftmaxPPL-$i.jld")  for i in 1:3],
    :SimplePPL100 => [load_jld("data/raw/ppl/SimplePPL-$i.jld")  for i in 1:3],
    :Softmax1 => [load_jld("data/raw/ppl/SoftmaxPPL-every-$i.jld")  for i in 1:3],
    :SimplePPL1 => [load_jld("data/raw/ppl/SimplePPL-every-$i.jld")  for i in 1:3],
)

Dict{Symbol, Vector} with 11 entries:
  :Softmax1     => RunData[RunData("SoftmaxPPL-every-1", :Softmax, ModelData(Dy…
  :None100P     => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :SimplePPL100 => RunData[RunData("SimplePPL-1", :SimplePPL, ModelData(Dynamic…
  :Medium100    => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :Simple100    => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :None100      => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :Simple1      => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :Medium1      => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :SimplePPL1   => RunData[RunData("SimplePPL-every-1", :SimplePPL, ModelData(D…
  :None1P       => PyObject[PyObject <dbn_ppl_rl.models.dbn.evaluate.thompson_s…
  :Softmax100   => RunData[RunData("SoftmaxPPL-1", :Softmax, ModelData(DynamicC…

In [10]:

function calculate_smoothing(mean_reward::Vector{Float64}, std_reward::Vector{Float64}, max_window_size::Int64)
    n = length(mean_reward)
    smoothed_mean = mean_reward
    smoothed_std_low = mean_reward .- std_reward
    smoothed_std_high = mean_reward .+ std_reward

    for idx in 1:n
        # Determine the dynamic window size
        window_size = min(max_window_size, idx, n - idx + 1)
        half_window = div(window_size, 2)

        # Compute the start and end of the window
        start_idx = max(1, idx - half_window)
        end_idx = min(n, idx + half_window)

        # Calculate the smoothed values for the current window
        window_mean = mean_reward[start_idx:end_idx]
        window_std = std_reward[start_idx:end_idx]
        smoothed_mean[idx] = mean(window_mean)
        smoothed_std_low[idx] = smoothed_mean[idx] - mean(window_std)
        smoothed_std_high[idx] = smoothed_mean[idx] + mean(window_std)
    end

    return smoothed_mean, smoothed_std_low, smoothed_std_high
end

function process_runs(results, max_window_size, is_100)
    means = Dict()

    for (key, run) in results
        if is_100 && !occursin("100", String(key)) || !is_100 && occursin("100", String(key))
            continue
        end

        # Extract and align keys across models
        all_keys = [sort(collect(keys(model.mean_rewards))) for model in run]
        min_keys = minimum(length.(all_keys))
        ks = all_keys[1][1:min_keys]

        filled_rewards = []
        all_rewards = []

        # Process individual runs
        for model in run
            rewards = [model.mean_rewards[ks[1]]]  # Start with the first key's reward
            for i in 2:min_keys
                k = ks[i]
                mult_factor = k - ks[i - 1]
                # Fill rewards over 100 steps for 100-types
                push!(rewards, fill(model.mean_rewards[k], mult_factor)...)
            end
            # Add filled rewards and original non-filled rewards
            push!(filled_rewards, rewards)
            push!(all_rewards, [model.mean_rewards[k] for k in ks])
        end

        # Compute mean and standard deviation over non-filled rewards
        mean_rewards = mean(hcat(all_rewards...), dims=2)[:]
        std_rewards = std(hcat(all_rewards...), dims=2)[:]

        # Dynamically smooth the means and std
        smoothed_mean, smoothed_std_low, smoothed_std_high = calculate_smoothing(mean_rewards, std_rewards, max_window_size)

        # Save to structure
        means[key] = MeansType(
            filled_rewards,
            mean(hcat(filled_rewards...), dims=2)[:],
            smoothed_mean,
            smoothed_std_low,
            smoothed_std_high,
            ks
        )
    end
    return means
end


process_runs (generic function with 1 method)

In [11]:
means_1 = process_runs(ts, 100, false)
means_100 = process_runs(ts, 6, true)

Dict{Any, Any} with 6 entries:
  :None100P     => MeansType([[-0.67594, -0.62842, -0.68253, -0.63209, -0.63209…
  :SimplePPL100 => MeansType([[-0.62715, -0.36589, -0.42002, -0.42002, -0.54821…
  :Medium100    => MeansType([[-0.70404, -0.66129, -0.62742, -0.62742, -0.69871…
  :Simple100    => MeansType([[-0.67862, -0.67333, -0.62492, -0.62492, -0.60324…
  :None100      => MeansType([[-0.68426, -0.7111, -0.65799, -0.65799, -0.6539, …
  :Softmax100   => MeansType([[-0.71519, -0.7079, -0.73134, -0.73134, -0.79105,…

In [12]:
using JSON3

function save_means_to_json(means, directory)
    for (key, value) in means
        println(key)
        file_path = joinpath(directory, "$(key).json")
        json_data = Dict(
            "individual_runs" => value.individual_runs,
            "mean_rewards" => value.mean_rewards,
            "smoothed_mean" => value.smoothed_mean,
            "smoothed_std_low" => value.smoothed_std_low,
            "smoothed_std_high" => value.smoothed_std_high,
            "keys_of_smoothed" => value.keys_of_smoothed,
            "name"=> key,
            "info" => Dict(),
        )
        JSON3.write(file_path, json_data)
    end
end

# Save means to JSON files in the "means_json" directory
save_means_to_json(means_1, "json/ts/")
save_means_to_json(means_100, "json/ts/")

Softmax1


SystemError: SystemError: opening file "json/ts/Softmax1.json": No such file or directory